Imports

In [17]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import math
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier 

Read Datafile

In [15]:
file = open("./data/SMSSpamCollection.txt")
labels=[]
sms=[]
while True:
	content=file.readline().lower()
	if not content:
		break
	sms.append(content[4:].replace("\n", ""))
	labels.append(content[:4].replace("\t", ""))
file.close()
sms=[s.replace("\t", "") for s in sms]

5574


Generate TF-IDF Values

In [8]:
ps=PorterStemmer()
stopWords=set(stopwords.words("english"))
idf={}
tfidf_temp=[]
tfidf=[]
for sm in sms:
    words=word_tokenize(sm)
    wc=0
    tf={}
    for word in words:
        word=ps.stem(word)
        if word in stopWords:
            continue
        wc+=1
        if word in tf:
            tf[word]+=1
        else:
            tf[word]=1
    for key in tf:
        tf[key]/=wc
        if key in idf:
            idf[key]+=1
        else:
            idf[key]=1
    tfidf_temp.append(tf)
for key in idf:
    idf[key]=math.log(len(sms)/idf[key])

for x in tfidf_temp:
    for key in idf:
        if key in x:
            x[key]*=idf[key]
        else:
            x[key]=0
    myKeys = list(x.keys())
    myKeys.sort()
    sorted_dict = {i: x[i] for i in myKeys}
    tfidf.append(sorted_dict)


In [14]:
tfidf_matrix=[]
for x in tfidf:
    mat_row=[]
    for key in x:
        mat_row.append(x[key])
    tfidf_matrix.append(mat_row)
file=open("TF-IDF Matrix.txt", 'w')
for items in tfidf_matrix:
    for item in items:
        stringer=str(item)+" "
        file.write(stringer)
    file.write("\n")
file.close()
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, labels, random_state=109, test_size=0.1, shuffle=True)
print(len(X_train))
print(len(X_test))

5016
558


KNN Classifier

In [23]:
knn=KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

0.9283154121863799
